# Data Science Capstone Project 

## Battle of the Neighborhoods - Singapore: Easties vs Westies 

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import math
import requests
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
!pip install folium
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Import StandardScaler required to normalize the dataset
from sklearn.preprocessing import StandardScaler

print('Libraries imported')

     |████████████████████████████████| 93 kB 2.8 MB/s  eta 0:00:01
Libraries imported


### Import neighborhood data

In [2]:
# The code was removed by Watson Studio for sharing.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Neighborhood  18 non-null     object 
 1   Region        18 non-null     object 
 2   Area          18 non-null     float64
 3   Population    18 non-null     int64  
 4   Density       18 non-null     float64
 5   Latitude      18 non-null     float64
 6   Longitude     18 non-null     float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.1+ KB


In [3]:
# Drop rows where population = 0
df_neighborhoods = df_neighborhoods[df_neighborhoods['Population']!=0].reset_index(drop=True)
df_neighborhoods

,Neighborhood,Region,Area,Population,Density,Latitude,Longitude
0,Boon Lay,West,8.23,30,3.6,1.3386,103.7058
1,Bukit Batok,West,11.13,139270,12513.0,1.3590,103.7637
2,Bukit Panjang,West,8.99,139030,15466.7,1.3774,103.7719
3,Choa Chu Kang,West,6.11,174330,28513.2,1.3840,103.7470
4,Clementi,West,9.49,91630,9650.3,1.3162,103.7649
5,Jurong East,West,17.83,84980,4766.9,1.3329,103.7436
6,Jurong West,West,14.69,272660,18563.5,1.3404,103.7090
7,Pioneer,West,12.10,100,8.3,1.3376,103.6974
8,Tengah,West,7.40,10,1.4,1.3555,103.7308
9,Tuas,West,30.04,70,2.3,1.2949,103.6305


### Get Venues Info using Foursquare API

#### Input API credentials

In [4]:
# Foursquare API credentials
CLIENT_ID = 'SDWG3TUM54SOW1LZBRZIPFONPSQQQNZLRCUJVWZ1FNALWWMT' # your Foursquare ID
CLIENT_SECRET = 'YXJN4X1TRESBVZZ24FHFAEFVGIUCA5XYLV1WYABFKYJGYAMJ' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: SDWG3TUM54SOW1LZBRZIPFONPSQQQNZLRCUJVWZ1FNALWWMT
CLIENT_SECRET:YXJN4X1TRESBVZZ24FHFAEFVGIUCA5XYLV1WYABFKYJGYAMJ


#### Create a function to get nearby venues 

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Use function to get the nearby venue information for all the neighborhoods in the df_neighborhoods

In [6]:
names = df_neighborhoods['Neighborhood']
latitudes = df_neighborhoods['Latitude']
longitudes = df_neighborhoods['Longitude']

df_venues = getNearbyVenues(names, latitudes, longitudes)
print(df_venues.shape)
df_venues.head()

Boon Lay
Bukit Batok
Bukit Panjang
Choa Chu Kang
Clementi
Jurong East
Jurong West
Pioneer
Tengah
Tuas
Western Water Catchment
Bedok
Changi
Pasir Ris
Paya Lebar
Tampines
(935, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Boon Lay,1.3386,103.7058,Carl's Jr.,1.340575,103.706384,Fast Food Restaurant
1,Boon Lay,1.3386,103.7058,Subway,1.340278,103.706548,Sandwich Place
2,Boon Lay,1.3386,103.7058,Din Tai Fung 鼎泰豐 (Din Tai Fung),1.339029,103.705765,Chinese Restaurant
3,Boon Lay,1.3386,103.7058,Ya Kun Kaya Toast,1.339619,103.705786,Breakfast Spot
4,Boon Lay,1.3386,103.7058,Starbucks,1.338906,103.705642,Coffee Shop


In [7]:
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bedok,98,98,98,98,98,98
Boon Lay,70,70,70,70,70,70
Bukit Batok,50,50,50,50,50,50
Bukit Panjang,47,47,47,47,47,47
Changi,40,40,40,40,40,40
Choa Chu Kang,44,44,44,44,44,44
Clementi,92,92,92,92,92,92
Jurong East,95,95,95,95,95,95
Jurong West,85,85,85,85,85,85


## Most common venue categories in the neighborhoods

#### One-Hot encode the venues category to flatten the DataFrame

In [8]:
# one hot encoding
venues_cat_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_cat_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# Merge region to venues_cat_onehot
regions = df_neighborhoods[['Neighborhood', 'Region']]
venues_cat_onehot = venues_cat_onehot.merge(regions, how='inner', on='Neighborhood')

# move neighborhood column to the first column
fixed_columns = ['Neighborhood', 'Region'] + list(venues_cat_onehot.columns[:-2])
venues_cat_onehot = venues_cat_onehot[fixed_columns]

venues_cat_onehot.head()

,Neighborhood,Region,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apres Ski Bar,...,Thrift / Vintage Store,Track,Trail,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wings Joint
0,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group by neighborhood and calculate weightage of each category by taking the mean of the frequency of occurrence of each category

In [9]:
neigh_venues_cat_grouped = venues_cat_onehot.groupby(['Neighborhood', 'Region']).mean().reset_index()
neigh_venues_cat_grouped.head()

,Neighborhood,Region,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apres Ski Bar,...,Thrift / Vintage Store,Track,Trail,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wings Joint
0,Bedok,East,0.000,0.00,0.000,0.000,0.0,0.00,0.010204,0.00,...,0.010204,0.0,0.000000,0.000,0.020408,0.0,0.00,0.0,0.0,0.010204
1,Boon Lay,West,0.000,0.00,0.000,0.000,0.0,0.00,0.014286,0.00,...,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.00,0.0,0.0,0.014286
2,Bukit Batok,West,0.000,0.00,0.000,0.000,0.0,0.00,0.020000,0.02,...,0.020000,0.0,0.020000,0.000,0.000000,0.0,0.02,0.0,0.0,0.000000
3,Bukit Panjang,West,0.000,0.00,0.000,0.000,0.0,0.00,0.021277,0.00,...,0.000000,0.0,0.021277,0.000,0.000000,0.0,0.00,0.0,0.0,0.000000
4,Changi,East,0.025,0.05,0.025,0.075,0.1,0.05,0.000000,0.00,...,0.000000,0.0,0.000000,0.025,0.000000,0.0,0.00,0.0,0.0,0.000000


#### Define a function to sort the venues

In [10]:
def return_most_common_venues(row, num_top_venues, col_num):
    row_categories = row.iloc[col_num:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Use function on each neighborhood to get the top 10 venue category for each neighborhood

In [11]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood', 'Region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neigh_venues_cat_grouped['Neighborhood']
neighborhoods_venues_sorted['Region'] = neigh_venues_cat_grouped['Region']

for ind in np.arange(neigh_venues_cat_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(neigh_venues_cat_grouped.iloc[ind, :], num_top_venues, 2)

neighborhoods_venues_sorted

,Neighborhood,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bedok,East,Chinese Restaurant,Food Court,Coffee Shop,Bakery,Supermarket,Asian Restaurant,Fast Food Restaurant,Noodle House,Japanese Restaurant,Sandwich Place
1,Boon Lay,West,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Dessert Shop,Coffee Shop,Indian Restaurant,Food Court,Convenience Store,Playground
2,Bukit Batok,West,Italian Restaurant,Coffee Shop,Supermarket,Historic Site,Indian Restaurant,Chinese Restaurant,Café,Bus Station,Shopping Mall,Gym / Fitness Center
3,Bukit Panjang,West,Food Court,Gym,Park,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Noodle House,Asian Restaurant,Supermarket,Shopping Mall
4,Changi,East,Coffee Shop,Airport Service,Airport Lounge,Airport,Airport Terminal,BBQ Joint,Fast Food Restaurant,Ice Cream Shop,Public Art,Noodle House
5,Choa Chu Kang,West,Fast Food Restaurant,Coffee Shop,Playground,Sushi Restaurant,Asian Restaurant,Food Court,Gym / Fitness Center,Café,Sandwich Place,Miscellaneous Shop
6,Clementi,West,Food Court,Bakery,Chinese Restaurant,Coffee Shop,Asian Restaurant,Dessert Shop,Dim Sum Restaurant,Supermarket,Gym,Café
7,Jurong East,West,Coffee Shop,Café,Japanese Restaurant,Chinese Restaurant,Food Court,Bus Station,Shopping Mall,Playground,Clothing Store,Multiplex
8,Jurong West,West,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Food Court,Dessert Shop,Playground,Indian Restaurant,Coffee Shop,Shopping Mall
9,Pasir Ris,East,Food Court,Fast Food Restaurant,Asian Restaurant,Pool,Park,Sushi Restaurant,Bus Line,Italian Restaurant,Coffee Shop,Playground


## Clustering Neighborhoods

#### Use k-means clustering to cluster neighborhoods

In [12]:
# Define number of clusters
k_clusters = 6

# Drop neighborhood column from neigh_venues_cat_grouped
neighborhoods_venues_clustering = neigh_venues_cat_grouped.drop(['Neighborhood', 'Region'], axis=1)

# instantiate kmeans model and cluster neighborhood
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(neighborhoods_venues_clustering)

# Print cluster labels for all neighborhoods
kmeans.labels_

array([1, 0, 1, 2, 5, 2, 0, 1, 0, 2, 0, 0, 1, 0, 4, 3], dtype=int32)

#### Add cluster labels to DataFrame containing the top 10 venue categories 

In [13]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#### Add coordinates to the DataFrame contain top 10 venue categories and cluster labels

In [14]:
# Merge coordinates data and cluster labels into a new dataframe
neighborhoods_venues_merged = neighborhoods_venues_sorted.merge(df_neighborhoods[['Neighborhood', 'Latitude', 'Longitude']], on='Neighborhood', how='left').set_index('Neighborhood', drop=True)
neighborhoods_venues_merged

,Cluster Labels,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
Neighborhood,,,,,,,,,,,,,,
Bedok,1,East,Chinese Restaurant,Food Court,Coffee Shop,Bakery,Supermarket,Asian Restaurant,Fast Food Restaurant,Noodle House,Japanese Restaurant,Sandwich Place,1.3236,103.9273
Boon Lay,0,West,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Dessert Shop,Coffee Shop,Indian Restaurant,Food Court,Convenience Store,Playground,1.3386,103.7058
Bukit Batok,1,West,Italian Restaurant,Coffee Shop,Supermarket,Historic Site,Indian Restaurant,Chinese Restaurant,Café,Bus Station,Shopping Mall,Gym / Fitness Center,1.3590,103.7637
Bukit Panjang,2,West,Food Court,Gym,Park,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Noodle House,Asian Restaurant,Supermarket,Shopping Mall,1.3774,103.7719
Changi,5,East,Coffee Shop,Airport Service,Airport Lounge,Airport,Airport Terminal,BBQ Joint,Fast Food Restaurant,Ice Cream Shop,Public Art,Noodle House,1.3450,103.9832
Choa Chu Kang,2,West,Fast Food Restaurant,Coffee Shop,Playground,Sushi Restaurant,Asian Restaurant,Food Court,Gym / Fitness Center,Café,Sandwich Place,Miscellaneous Shop,1.3840,103.7470
Clementi,0,West,Food Court,Bakery,Chinese Restaurant,Coffee Shop,Asian Restaurant,Dessert Shop,Dim Sum Restaurant,Supermarket,Gym,Café,1.3162,103.7649
Jurong East,1,West,Coffee Shop,Café,Japanese Restaurant,Chinese Restaurant,Food Court,Bus Station,Shopping Mall,Playground,Clothing Store,Multiplex,1.3329,103.7436
Jurong West,0,West,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Food Court,Dessert Shop,Playground,Indian Restaurant,Coffee Shop,Shopping Mall,1.3404,103.7090


#### Create map for visualization of the neighborhood clustering

In [15]:
# create map
sg_latitude = 1.3521
sg_longitude = 103.8198
map_clusters = folium.Map(location=[sg_latitude, sg_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_merged['Latitude'], neighborhoods_venues_merged['Longitude'] \
                                  , neighborhoods_venues_merged.index, neighborhoods_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Deep-diving into the clusters characteristics

In [16]:
# Label the onehot-encoded dataframe with cluster label
df_neigh_cluster = neighborhoods_venues_sorted[['Neighborhood', 'Cluster Labels']]
venues_cat_onehot_labelled = venues_cat_onehot.merge(df_neigh_cluster, on='Neighborhood', how='left')
venues_cat_onehot_labelled.head()

,Neighborhood,Region,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apres Ski Bar,...,Track,Trail,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wings Joint,Cluster Labels
0,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boon Lay,West,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Group rows by the cluster label
cluster_venues_onehot = venues_cat_onehot_labelled.groupby('Cluster Labels').mean()
cluster_venues_onehot

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apres Ski Bar,Arts & Crafts Store,Asian Restaurant,...,Thrift / Vintage Store,Track,Trail,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wings Joint
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.00,0.000,0.000,0.0,0.00,0.007059,0.000000,0.002353,0.075294,...,0.000000,0.000000,0.002353,0.000,0.007059,0.002353,0.000000,0.002353,0.000000,0.007059
1,0.003185,0.00,0.000,0.000,0.0,0.00,0.009554,0.003185,0.003185,0.015924,...,0.006369,0.003185,0.006369,0.000,0.009554,0.000000,0.003185,0.000000,0.000000,0.003185
2,0.000000,0.00,0.000,0.000,0.0,0.00,0.006803,0.000000,0.000000,0.054422,...,0.000000,0.000000,0.006803,0.000,0.000000,0.000000,0.000000,0.000000,0.006803,0.000000
3,0.000000,0.00,0.000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00,0.000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.025000,0.05,0.025,0.075,0.1,0.05,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Results section

In [18]:
# List top 10 common venue categories in each cluster
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cluster_venues_top10 = pd.DataFrame(columns=columns)
cluster_venues_top10['Cluster'] = cluster_venues_onehot.index

for ind in np.arange(cluster_venues_onehot.shape[0]):
    cluster_venues_top10.iloc[ind, 1:] = return_most_common_venues(cluster_venues_onehot.iloc[ind, :], num_top_venues, 1)

cluster_venues_top10

,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Asian Restaurant,Chinese Restaurant,Food Court,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Dessert Shop,Indian Restaurant,Supermarket,Grocery Store
1,1,Coffee Shop,Chinese Restaurant,Bus Station,Food Court,Café,Japanese Restaurant,Fast Food Restaurant,Shopping Mall,Playground,Supermarket
2,2,Fast Food Restaurant,Food Court,Coffee Shop,Asian Restaurant,Park,Gym,Sushi Restaurant,Playground,Pool,Sandwich Place
3,3,Food Court,Bus Station,Flower Shop,Gun Range,Wings Joint,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Service
4,4,Food Court,Gas Station,Food Truck,Asian Restaurant,Wings Joint,Filipino Restaurant,Fried Chicken Joint,French Restaurant,Food Service,Flower Shop
5,5,Airport Service,Coffee Shop,Airport Lounge,Airport,Airport Terminal,BBQ Joint,Fast Food Restaurant,Convenience Store,Sandwich Place,Public Art
